<a href="https://colab.research.google.com/github/olaviinha/NeuralTextToImage/blob/main/CLIP%2BVQGAN_crowsonkb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font face="Trebuchet MS" size="6">Big Sleep: CLIP+VQGAN <font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><font color="#999" size="4">CLIP+VQGAN: Neural text-to-image</font><font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><a href="https://github.com/olaviinha/NeuralImageGeneration" target="_blank"><font color="#999" size="4">Github</font></a>

Big Sleep generates images from text input. It's originally a combination of [CLIP](https://github.com/openai/CLIP) by OpenAI and [BigGAN](https://arxiv.org/abs/1809.11096) by Andrew Brock et al., a concept introduced by [Ryan Murdock](https://github.com/rynmurdock) in his [original notebook](https://colab.research.google.com/drive/1NCceX2mbiKOSlAd_o7IU7nA9UskKN5WR?usp=sharing). This notebook is based on an early (and spanish) version of a [VQGAN](https://arxiv.org/abs/2012.09841)+CLIP notebook by [Katherine Crowson](https://github.com/crowsonkb).

<hr size="1" color="#666">

### Tips:
- Enter a simple string of text to `generate_image_of` field. Advanced usage: You may also 1) use a semicolon `;` as a separator to batch process multiple strings of texts to images in one go, 2) use a pipe `|` to train the image on multiple strings of text. If field is left empty, a random blog headline will be used.
- Enter `output_dir` path relative to your Google Drive root, or leave blank to not save output anywhere outside this notebook. Each run of the _Sleep_ cell will **create a new subdirectory** under `output_dir`, under which all material will be saved.
- `initial_image` and `target_image` define the "shape" and "texture" of the result image; best way to understand what they do is just to test them out. Field values may be URL addresses to images online or paths to images located in your Google Drive (enter path relative to Drive root).
- `random_initial_photo` and `random_target_photo` will fetch random photographs from [Lorem Picsum](https://picsum.photos/).
- You may find that over 400 iterations is usually a waste of time.
- If you get _CUDA out of memory_ errors, reduce width and/or height. If it still happens (when you know it shouldn't), factory reset runtime). Colab is unable to generate images larger than ~0.5 MP, or 700x700 pixels. Some standard aspect ratios as maximum resolutions that Colab Pro can handle (from the original notebook):‎‎‎‎‏‏‎

 ‎‏‏‎ ‎‏‏‎ ‎‏‏‎ ‎‏‏‎ 1:1 = 700 x 700‎‎‎‎‏‏‎ ‎‏‏‎ ‎‏‏‎ ‎‏‏‎ ‎‏‏‎ ‎4:3 = 808 x 606‎‎‎‎‏‏‎ ‎‏‏‎ ‎‏‏‎ ‎‏‏‎ ‎‏‏‎ 16:9 = 928 x 522‎‎‎‎‏‏‎ ‎‏‏‎ ‎‏‏‎ ‎‏‏‎ ‎‏‏‎ 2:1 = 988 x 494‎‎‎‎‏‏‎ ‎‏‏‎ ‎‏‏‎ ‎‏‏‎ ‎‏‏‎ 1.66:1 = 903 x 544


In [ ]:
#@title #Setup
#@markdown This cell needs to be run only once. It will mount your Google Drive and setup prerequisites.

force_setup = False
pip_packages = 'ftfy regex tqdm omegaconf pytorch-lightning kornia einops stegano python-xmp-toolkit imgtag imageio-ffmpeg transformers pillow==7.1.2'
main_repository = ''

import os
from google.colab import output
import warnings
warnings.filterwarnings('ignore')
%cd /content/

# inhagcutils
if not os.path.isfile('/content/inhagcutils.ipynb') or force_setup == True:
  !pip -q install import-ipynb {pip_packages}
  !apt -q install exempi
  !curl -s -O https://raw.githubusercontent.com/olaviinha/inhagcutils/master/inhagcutils.ipynb
import import_ipynb
from inhagcutils import *

# Mount Drive
if not os.path.isdir('/content/drive') and force_setup == False:
  from google.colab import drive
  drive.mount('/content/drive')

# Drive symlink
if not os.path.isdir('/content/mydrive') and force_setup == False:
  os.symlink('/content/drive/My Drive', '/content/mydrive')
  drive_root_set = True
drive_root = '/content/mydrive/'

# !git clone {main_repository}

#-----------------------------------
 
!git clone https://github.com/openai/CLIP                 &> /dev/null
!git clone https://github.com/CompVis/taming-transformers &> /dev/null

# !pip install ftfy regex tqdm omegaconf pytorch-lightning  &> /dev/null
# !pip install kornia                                       &> /dev/null
# !pip install einops                                       &> /dev/null
# !pip install stegano                                      &> /dev/null
# !apt install exempi                                       &> /dev/null
# !pip install python-xmp-toolkit                           &> /dev/null
# !pip install imgtag                                       &> /dev/null
# !pip install pillow==7.1.2                                &> /dev/null
 
# !pip install imageio-ffmpeg                               &> /dev/null
# !pip install transformers                                 &> /dev/null

downloaded_models = []

#-----------------------------------

# @title Carga de bibliotecas y definiciones
 
import argparse
import math
from pathlib import Path
import sys
import requests
 
sys.path.append('./taming-transformers')
from IPython import display
from base64 import b64encode
from omegaconf import OmegaConf
from PIL import Image
from taming.models import cond_transformer, vqgan
import torch
from torch import nn, optim
from torch.nn import functional as F
from torchvision import transforms
from torchvision.transforms import functional as TF
from tqdm.notebook import tqdm
 
from CLIP import clip
import kornia.augmentation as K
import numpy as np
import imageio
from PIL import ImageFile, Image
from imgtag import ImgTag    # metadatos 
from libxmp import *         # metadatos
import libxmp                # metadatos
from stegano import lsb
import json
from urllib.parse import urljoin, urlparse
import itertools
ImageFile.LOAD_TRUNCATED_IMAGES = True

def keyboardInterruptHandler():
  global dir_output, uniq_id
  op(c.warn, 'Interrupted!', 'Cleaning up...')
  remove_dirs([dir_output])
  print('Run', uniq_id, 'directory and content removed:', dir_output)
  sys.exit()
    
def sinc(x):
    return torch.where(x != 0, torch.sin(math.pi * x) / (math.pi * x), x.new_ones([]))
 
 
def lanczos(x, a):
    cond = torch.logical_and(-a < x, x < a)
    out = torch.where(cond, sinc(x) * sinc(x/a), x.new_zeros([]))
    return out / out.sum()
 
 
def ramp(ratio, width):
    n = math.ceil(width / ratio + 1)
    out = torch.empty([n])
    cur = 0
    for i in range(out.shape[0]):
        out[i] = cur
        cur += ratio
    return torch.cat([-out[1:].flip([0]), out])[1:-1]
 
def fix_path(path, add_slash=False):
  if path.endswith('/'):
    path = path #path[:-1]
  if not path.endswith('/'):
    path = path+"/"
  if path.startswith('/') and add_slash == True:
    path = path[1:]
  return path

def resample(input, size, align_corners=True):
    n, c, h, w = input.shape
    dh, dw = size
 
    input = input.view([n * c, 1, h, w])
 
    if dh < h:
        kernel_h = lanczos(ramp(dh / h, 2), 2).to(input.device, input.dtype)
        pad_h = (kernel_h.shape[0] - 1) // 2
        input = F.pad(input, (0, 0, pad_h, pad_h), 'reflect')
        input = F.conv2d(input, kernel_h[None, None, :, None])
 
    if dw < w:
        kernel_w = lanczos(ramp(dw / w, 2), 2).to(input.device, input.dtype)
        pad_w = (kernel_w.shape[0] - 1) // 2
        input = F.pad(input, (pad_w, pad_w, 0, 0), 'reflect')
        input = F.conv2d(input, kernel_w[None, None, None, :])
 
    input = input.view([n, c, h, w])
    return F.interpolate(input, size, mode='bicubic', align_corners=align_corners)
 
 
class ReplaceGrad(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x_forward, x_backward):
        ctx.shape = x_backward.shape
        return x_forward
 
    @staticmethod
    def backward(ctx, grad_in):
        return None, grad_in.sum_to_size(ctx.shape)
 
 
replace_grad = ReplaceGrad.apply
 
 
class ClampWithGrad(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, min, max):
        ctx.min = min
        ctx.max = max
        ctx.save_for_backward(input)
        return input.clamp(min, max)
 
    @staticmethod
    def backward(ctx, grad_in):
        input, = ctx.saved_tensors
        return grad_in * (grad_in * (input - input.clamp(ctx.min, ctx.max)) >= 0), None, None
 
 
clamp_with_grad = ClampWithGrad.apply
 
 
def vector_quantize(x, codebook):
    d = x.pow(2).sum(dim=-1, keepdim=True) + codebook.pow(2).sum(dim=1) - 2 * x @ codebook.T
    indices = d.argmin(-1)
    x_q = F.one_hot(indices, codebook.shape[0]).to(d.dtype) @ codebook
    return replace_grad(x_q, x)
 
 
class Prompt(nn.Module):
    def __init__(self, embed, weight=1., stop=float('-inf')):
        super().__init__()
        self.register_buffer('embed', embed)
        self.register_buffer('weight', torch.as_tensor(weight))
        self.register_buffer('stop', torch.as_tensor(stop))
 
    def forward(self, input):
        input_normed = F.normalize(input.unsqueeze(1), dim=2)
        embed_normed = F.normalize(self.embed.unsqueeze(0), dim=2)
        dists = input_normed.sub(embed_normed).norm(dim=2).div(2).arcsin().pow(2).mul(2)
        dists = dists * self.weight.sign()
        return self.weight.abs() * replace_grad(dists, torch.maximum(dists, self.stop)).mean()
 
 
def parse_prompt(prompt):
    vals = prompt.rsplit(':', 2)
    vals = vals + ['', '1', '-inf'][len(vals):]
    return vals[0], float(vals[1]), float(vals[2])
 
 
class MakeCutouts(nn.Module):
    def __init__(self, cut_size, cutn, cut_pow=1.):
        super().__init__()
        self.cut_size = cut_size
        self.cutn = cutn
        self.cut_pow = cut_pow
        self.augs = nn.Sequential(
            K.RandomHorizontalFlip(p=0.5),
            # K.RandomSolarize(0.01, 0.01, p=0.7),
            K.RandomSharpness(0.3,p=0.4),
            K.RandomAffine(degrees=30, translate=0.1, p=0.8, padding_mode='border'),
            K.RandomPerspective(0.2,p=0.4),
            K.ColorJitter(hue=0.01, saturation=0.01, p=0.7))
        self.noise_fac = 0.1
 
 
    def forward(self, input):
        sideY, sideX = input.shape[2:4]
        max_size = min(sideX, sideY)
        min_size = min(sideX, sideY, self.cut_size)
        cutouts = []
        for _ in range(self.cutn):
            size = int(torch.rand([])**self.cut_pow * (max_size - min_size) + min_size)
            offsetx = torch.randint(0, sideX - size + 1, ())
            offsety = torch.randint(0, sideY - size + 1, ())
            cutout = input[:, :, offsety:offsety + size, offsetx:offsetx + size]
            cutouts.append(resample(cutout, (self.cut_size, self.cut_size)))
        batch = self.augs(torch.cat(cutouts, dim=0))
        if self.noise_fac:
            facs = batch.new_empty([self.cutn, 1, 1, 1]).uniform_(0, self.noise_fac)
            batch = batch + facs * torch.randn_like(batch)
        return batch
 
 
def load_vqgan_model(config_path, checkpoint_path):
    config = OmegaConf.load(config_path)
    if config.model.target == 'taming.models.vqgan.VQModel':
        model = vqgan.VQModel(**config.model.params)
        model.eval().requires_grad_(False)
        model.init_from_ckpt(checkpoint_path)
    elif config.model.target == 'taming.models.cond_transformer.Net2NetTransformer':
        parent_model = cond_transformer.Net2NetTransformer(**config.model.params)
        parent_model.eval().requires_grad_(False)
        parent_model.init_from_ckpt(checkpoint_path)
        model = parent_model.first_stage_model
    else:
        raise ValueError(f'unknown model type: {config.model.target}')
    del model.loss
    return model
 
 
def resize_image(image, out_size):
    ratio = image.size[0] / image.size[1]
    area = min(image.size[0] * image.size[1], out_size[0] * out_size[1])
    size = round((area * ratio)**0.5), round((area / ratio)**0.5)
    return image.resize(size, Image.LANCZOS)

def synth(z):
    z_q = vector_quantize(z.movedim(1, 3), model.quantize.embedding.weight).movedim(3, 1)
    return clamp_with_grad(model.decode(z_q).add(1).div(2), 0, 1)

def add_xmp_data(nombrefichero):
    imagen = ImgTag(filename=nombrefichero)
    imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'creator', 'VQGAN+CLIP', {"prop_array_is_ordered":True, "prop_value_is_array":True})
    if args.prompts:
        imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'title', " | ".join(args.prompts), {"prop_array_is_ordered":True, "prop_value_is_array":True})
    else:
        imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'title', 'None', {"prop_array_is_ordered":True, "prop_value_is_array":True})
    imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'i', str(i), {"prop_array_is_ordered":True, "prop_value_is_array":True})
    imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'model', nombre_modelo, {"prop_array_is_ordered":True, "prop_value_is_array":True})
    imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'seed',str(seed) , {"prop_array_is_ordered":True, "prop_value_is_array":True})
    imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'input_images',str(input_images) , {"prop_array_is_ordered":True, "prop_value_is_array":True})
    #for frases in args.prompts:
    #    imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'Prompt' ,frases, {"prop_array_is_ordered":True, "prop_value_is_array":True})
    imagen.close()

def add_stegano_data(filename):
    data = {
        "title": " | ".join(args.prompts) if args.prompts else None,
        "notebook": "VQGAN+CLIP",
        "i": i,
        "model": nombre_modelo,
        "seed": str(seed),
        "input_images": input_images
    }
    lsb.hide(filename, json.dumps(data)).save(filename)

@torch.no_grad()
def checkin(i, losses):
    global dir_progress, dir_output, max_iteraciones, id, display_save_every
    losses_str = ', '.join(f'{loss.item():g}' for loss in losses)
    # tqdm.write(f'i: {i}, loss: {sum(losses).item():g}, losses: {losses_str}')
    out = synth(z)
    TF.to_pil_image(out[0].cpu()).save('progress.png')
    add_stegano_data('progress.png')
    add_xmp_data('progress.png')
    if display_save_every:
      display.display(display.Image('progress.png'))
    save_img = dir_progress+'step_'+str(i).zfill(4)+'.png'
    !cp progress.png {save_img}
    # if i is max_iteraciones:
    #   !cp progress.png {dir_output}{id}.png
    # else:
    prog_path = dir_progress.replace(drive_root, '')
    prog_img = prog_path+'step_'+str(i).zfill(4)+'.png'
    if display_save_every:
      op(c.ok, '^Image saved as', prog_img + '\n')
    else:
      op(c.ok, 'Image saved as', prog_img)

def ascend_txt():
    global i, dir_steps
    out = synth(z)
    iii = perceptor.encode_image(normalize(make_cutouts(out))).float()

    result = []

    if args.init_weight:
        result.append(F.mse_loss(z, z_orig) * args.init_weight / 2)

    for prompt in pMs:
        result.append(prompt(iii))
    img = np.array(out.mul(255).clamp(0, 255)[0].cpu().detach().numpy().astype(np.uint8))[:,:,:]
    img = np.transpose(img, (1, 2, 0))
    filename = f"{dir_steps}/{i:04}.png"
    imageio.imwrite(filename, np.array(img))
    add_stegano_data(filename)
    add_xmp_data(filename)
    return result

def train(i):
    opt.zero_grad()
    lossAll = ascend_txt()
    if i % args.display_freq == 0:
      checkin(i, lossAll)
    loss = sum(lossAll)
    loss.backward()
    opt.step()
    with torch.no_grad():
      z.copy_(z.maximum(z_min).minimum(z_max))

def append_txt(file, content):
  txt = open(txt_file, 'a+') 
  txt.writelines(content+'\n')
  txt.close();
#-----------------------------------

dir_tmp = '/content/tmp/'
dir_steps = '/content/tmp/steps/'
dir_initial = '/content/tmp/init/'
dir_target = '/content/tmp/target/'
create_dirs([dir_tmp, dir_steps, dir_initial, dir_target])

output.clear()
# !nvidia-smi
op(c.ok, 'Setup finished.')

In [ ]:
#@markdown <br>

#@markdown #S̛̞̩͎͓ ̦̤͉͚̏ ̧̠͋͘ͅl͕̞͕̝͗̐͘.̠̰̳̫̈́̚ ̡͉̼̩̬̈́̇͒͘ȩ̨͎͛̔͆͊̏͜ͅ.͕̩̹̠̕͜ ̛̦̦̮e̢͐͊͂̀̊ͅ ̜̙̝̊͋ ̬̝̱̱͗p̮̎̽̌

#@markdown <br>

torch.cuda.empty_cache()
clean_dirs([dir_steps, dir_initial, dir_target])

generate_image_of = "" #@param {type:"string"}
width = 928 #@param {type:"slider", min:512, max:1024, step:2}
height = 522 #@param {type:"slider", min:512, max:1024, step:2}
output_dir = '' #@param {type:"string"}

#@markdown <hr color="#666" size="1">
#@markdown <font size="1">&nbsp;</font>

#@markdown ### Advanced settings

model = "vqgan_imagenet_f16_16384" #@param ["vqgan_imagenet_f16_16384", "vqgan_imagenet_f16_1024", "wikiart_1024", "wikiart_16384", "coco", "faceshq", "sflckr"]
# model = "vqgan_imagenet_f16_16384" #@param ["vqgan_imagenet_f16_16384", "wikiart_16384"]

iterations = 400 #@param {type:"slider", min:0, max:2000, step:100}
save_every = 50 #@param {type:"slider", min:0, max:500, step:1}
display_save_every = True #@param {type:"boolean"}
initial_image = '' #@param {type:"string"}
random_initial_photo = True #@param {type:"boolean"}
target_image = '' #@param {type:"string"}
random_target_photo = False #@param {type:"boolean"}
create_video = False #@param {type:"boolean"}

save_all_steps = False
remove_interrupted = True
repetitions = 0

#
# --- Very advanced settings ---------------------------------

# #@markdown <hr color="#666" size="1">
# #@markdown <font size="1">&nbsp;</font>

# #@markdown ### Very advanced settings
# repetitions = 0 #@param {type:"slider", min:0, max:20, step:1}
# save_all_steps = False #@param {type:"boolean"}
# remove_interrupted = True #@param {type:"boolean"}

# --- //Very advanced settings -------------------------------
#

input_images = ""
text = generate_image_of
textos = generate_image_of
modelo = model
ancho = width
alto = height
imagen_inicial = initial_image
imagenes_objetivo = target_image
max_iteraciones = iterations
intervalo_imagenes = save_every
seed = -1
display_model = model

dlmsg = 'Downloading required model...\n'
if model is 'vqgan_imagenet_f16_1024' and 'vqgan_imagenet_f16_1024' not in downloaded_models:
  op(c.title, dlmsg)
  !curl -L -o vqgan_imagenet_f16_1024.yaml -C - 'http://mirror.io.community/blob/vqgan/vqgan_imagenet_f16_1024.yaml' #ImageNet 1024
  !curl -L -o vqgan_imagenet_f16_1024.ckpt -C - 'http://mirror.io.community/blob/vqgan/vqgan_imagenet_f16_1024.ckpt'  #ImageNet 1024
  downloaded_models.append('vqgan_imagenet_f16_1024')
  output.clear()
if model is 'vqgan_imagenet_f16_16384' and 'vqgan_imagenet_f16_16384' not in downloaded_models:
  op(c.title, dlmsg)
  !curl -L -o vqgan_imagenet_f16_16384.yaml -C - 'http://mirror.io.community/blob/vqgan/vqgan_imagenet_f16_16384.yaml' #ImageNet 16384
  !curl -L -o vqgan_imagenet_f16_16384.ckpt -C - 'http://mirror.io.community/blob/vqgan/vqgan_imagenet_f16_16384.ckpt' #ImageNet 16384
  downloaded_models.append('vqgan_imagenet_f16_16384')
  output.clear()
if model is 'coco' and 'coco' not in downloaded_models:
  op(c.title, dlmsg)
  !curl -L -o coco.yaml -C - 'https://dl.nmkd.de/ai/clip/coco/coco.yaml' #COCO
  !curl -L -o coco.ckpt -C - 'https://dl.nmkd.de/ai/clip/coco/coco.ckpt' #COCO
  downloaded_models.append('coco')
  output.clear()
if model is 'faceshq' and 'faceshq' not in downloaded_models:
  op(c.title, dlmsg)
  !curl -L -o faceshq.yaml -C - 'https://drive.google.com/uc?export=download&id=1fHwGx_hnBtC8nsq7hesJvs-Klv-P0gzT' #FacesHQ
  !curl -L -o faceshq.ckpt -C - 'https://app.koofr.net/content/links/a04deec9-0c59-4673-8b37-3d696fe63a5d/files/get/last.ckpt?path=%2F2020-11-13T21-41-45_faceshq_transformer%2Fcheckpoints%2Flast.ckpt' #FacesHQ
  downloaded_models.append('faceshq')
  output.clear()
if model is 'wikiart_1024 'and 'wikiart_1024' not in downloaded_models:
  op(c.title, dlmsg)
  !curl -L -o wikiart_1024.yaml -C - 'http://mirror.io.community/blob/vqgan/wikiart.yaml' #WikiArt 1024
  !curl -L -o wikiart_1024.ckpt -C - 'http://mirror.io.community/blob/vqgan/wikiart.ckpt' #WikiArt 1024
  downloaded_models.append('wikiart_1024')
  output.clear()
if model is 'wikiart_16384' and 'wikiart_16384' not in downloaded_models:
  op(c.title, dlmsg)
  !curl -L -o wikiart_16384.yaml -C - 'http://mirror.io.community/blob/vqgan/wikiart_16384.yaml' #WikiArt 16384
  !curl -L -o wikiart_16384.ckpt -C - 'http://mirror.io.community/blob/vqgan/wikiart_16384.ckpt' #WikiArt 16384
  downloaded_models.append('wikiart_16384')
  output.clear()
if model is 'sflckr' and 'sflckr' not in downloaded_models:
  op(c.title, dlmsg)
  !curl -L -o sflckr.yaml -C - 'https://heibox.uni-heidelberg.de/d/73487ab6e5314cb5adba/files/?p=%2Fconfigs%2F2020-11-09T13-31-51-project.yaml&dl=1' #S-FLCKR
  !curl -L -o sflckr.ckpt -C - 'https://heibox.uni-heidelberg.de/d/73487ab6e5314cb5adba/files/?p=%2Fcheckpoints%2Flast.ckpt&dl=1' #S-FLCKR
  downloaded_models.append('sflckr')
  output.clear()

op(c.title, 'Majik & wizardry...\n')

nombres_modelos = {
  "vqgan_imagenet_f16_16384": "ImageNet 16384",
  "vqgan_imagenet_f16_1024":"ImageNet 1024", 
  "wikiart_1024":"WikiArt 1024",
  "wikiart_16384":"WikiArt 16384",
  "coco":"COCO-Stuff",
  "faceshq":"FacesHQ",
  "sflckr":"S-FLCKR"
}
nombre_modelo = nombres_modelos[modelo]     

if output_dir is '' or not output_dir:
  drive_root = '/content/fauxdrive/'
  output_dir = 'output'
  create_dirs([drive_root, output_dir])
else:
  drive_root = '/content/mydrive/'

if text is not '':
  if check_input_type(drive_root+text) is 'file':
    input_txt = drive_root+text
    with open(input_txt) as f:
      texts = f.readlines()
      texts = [x.strip() for x in texts] 
  elif ";" in text:
    texts = text.split(';')
    texts = [text.strip() for text in texts]
  else:
    texts = [text]
else:
  text = requests.get('https://api.inha.asia/headline/').text
  texts = [text]
if repetitions > 0:
  texts = list(itertools.chain.from_iterable(itertools.repeat(x, repetitions) for x in texts))

def remove_params(url):
  return urljoin(url, urlparse(url).path)

if seed == -1:
  seed = None

# ------------------------------------------

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
# if textos:
#     print('Using texts:', textos)

uniq_id = gen_id()
repeat_index = 1

for textos in texts:

  imagen_inicial = initial_image
  imagenes_objetivo = target_image

  if random_initial_photo is True:
    imagen_inicial = 'https://picsum.photos/'+str(width)+'/'+str(height)
  if random_target_photo is True:
    imagen_inicial = 'https://picsum.photos/'+str(width)+'/'+str(height)
  if imagen_inicial == '' or not imagen_inicial:
    imagen_inicial = None
  else:
    if check_input_type(imagen_inicial) is "link":
      imagen_inicial = remove_params(imagen_inicial)
      ext = path_ext(imagen_inicial) or '.jpg'
      new_initial = dir_initial+'initial'+ext
      !wget -O {new_initial} {imagen_inicial}
      imagen_inicial = new_initial
    else:
      imagen_inicial = drive_root+imagen_inicial
      
  if imagenes_objetivo == '' or not imagenes_objetivo:
    imagenes_objetivo = []
  elif check_input_type(imagenes_objetivo) is "link":
    imagenes_objetivo = imagenes_objetivo.split("|")
    for i, img in enumerate(imagenes_objetivo):
      new_target = dir_target+path_leaf(remove_params(img))
      !wget -O {new_target} {img}
      imagenes_objetivo[i] = new_target
  else:
    if ";" in imagenes_objetivo:
      imgs = imagenes_objetivo.split(';')
      imagenes_objetivo = [drive_root+image.strip() for image in imags]
    else:
      imagenes_objetivo = [drive_root+imagenes_objetivo]
  if imagen_inicial or imagenes_objetivo != []:
      input_images = True

  print( imagenes_objetivo )

  if save_all_steps is False:
    clean_dirs([dir_steps])

  display_text = textos
  title = textos.split("|")[0].title()
  file_title = ''.join(e for e in title if e.isalnum())
  textos = [frase.strip() for frase in textos.split("|")]

  id = uniq_id+'_'+file_title
  if repetitions > 0:
    id = uniq_id+'_'+str(repeat_index)+'_'+file_title

  dir_output = fix_path(drive_root+output_dir)+id+'/'
  dir_progress = dir_output+'progress/'
  create_dirs([dir_output, dir_progress])

  if save_all_steps is True:
    dir_steps = dir_output+'steps/'
    create_dirs([dir_steps])

  if textos == ['']:
    textos = []

  args = argparse.Namespace(
      prompts=textos,
      image_prompts=imagenes_objetivo,
      noise_prompt_seeds=[],
      noise_prompt_weights=[],
      size=[ancho, alto],
      init_image=imagen_inicial,
      init_weight=0.,
      clip_model='ViT-B/32',
      vqgan_config=f'{modelo}.yaml',
      vqgan_checkpoint=f'{modelo}.ckpt',
      step_size=0.1,
      cutn=64,
      cut_pow=1.,
      display_freq=intervalo_imagenes,
      seed=seed,
  )

  if args.seed is None:
      seed = torch.seed()
  else:
      seed = args.seed
  torch.manual_seed(seed)
  print('Using seed:', seed)

  # ------------------------------------------------

  print( args.vqgan_config )
  print( args.vqgan_checkpoint )
  
  model = load_vqgan_model(args.vqgan_config, args.vqgan_checkpoint).to(device)
  perceptor = clip.load(args.clip_model, jit=False)[0].eval().requires_grad_(False).to(device)

  cut_size = perceptor.visual.input_resolution
  e_dim = model.quantize.e_dim
  f = 2**(model.decoder.num_resolutions - 1)
  make_cutouts = MakeCutouts(cut_size, args.cutn, cut_pow=args.cut_pow)
  n_toks = model.quantize.n_e
  toksX, toksY = args.size[0] // f, args.size[1] // f
  sideX, sideY = toksX * f, toksY * f
  z_min = model.quantize.embedding.weight.min(dim=0).values[None, :, None, None]
  z_max = model.quantize.embedding.weight.max(dim=0).values[None, :, None, None]

  if args.init_image:
      pil_image = Image.open(args.init_image).convert('RGB')
      pil_image = pil_image.resize((sideX, sideY), Image.LANCZOS)
      z, *_ = model.encode(TF.to_tensor(pil_image).to(device).unsqueeze(0) * 2 - 1)
  else:
      one_hot = F.one_hot(torch.randint(n_toks, [toksY * toksX], device=device), n_toks).float()
      z = one_hot @ model.quantize.embedding.weight
      z = z.view([-1, toksY, toksX, e_dim]).permute(0, 3, 1, 2)
  z_orig = z.clone()
  z.requires_grad_(True)
  opt = optim.Adam([z], lr=args.step_size)

  normalize = transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                                  std=[0.26862954, 0.26130258, 0.27577711])

  pMs = []

  for prompt in args.prompts:
      txt, weight, stop = parse_prompt(prompt)
      embed = perceptor.encode_text(clip.tokenize(txt).to(device)).float()
      pMs.append(Prompt(embed, weight, stop).to(device))

  for prompt in args.image_prompts:
    print( prompt )
    path, weight, stop = parse_prompt(prompt)
    print( path )
    img = resize_image(Image.open(path).convert('RGB'), (sideX, sideY))
    batch = make_cutouts(TF.to_tensor(img).unsqueeze(0).to(device))
    embed = perceptor.encode_image(normalize(batch)).float()
    pMs.append(Prompt(embed, weight, stop).to(device))

  for seed, weight in zip(args.noise_prompt_seeds, args.noise_prompt_weights):
      gen = torch.Generator().manual_seed(seed)
      embed = torch.empty([1, perceptor.visual.output_dim]).normal_(generator=gen)
      pMs.append(Prompt(embed, weight).to(device))

  #-----------------------------------

  txt_file = dir_output+uniq_id+'-info.txt'
  txt = open(txt_file,'w') 
  params = [timestamp()+' '+uniq_id+'\n\n',
            'model:         '+display_model+'\n',
            'iterations:    '+str(iterations)+'\n',
            'text:          '+text+'\n',
            'initial_image: '+initial_image+'\n',
            'target_image:  '+target_image+'\n']
  txt.writelines(params)
  txt.close()

  output.clear()
  op(c.title, '\nGenerating image of:', display_text)
  op(c.title, 'Run ID:', uniq_id)
  if repetitions > 0:
    op(c.title, 'Repetition:', repeat_index)
  op(c.okb, 'Sweet dreams.\n')

  i = 0
  try:
    with tqdm() as pbar:
      while True:
        train(i)
        if i == max_iteraciones:
          break
        i += 1
        pbar.update()
  except KeyboardInterrupt:
    if remove_interrupted: keyboardInterruptHandler()

  last_step = dir_steps+str(i).zfill(4)+'.png'
  fin_out = dir_output+file_title+'.png'
  !cp {last_step} {fin_out}
  display_fin = fin_out.replace(drive_root, '')
  op(c.ok, 'Final image saved as', display_fin)

  # ---------------------------------------------------------
  # - video

  if create_video is True:
    op(c.title, '\nGenerating video')

    init_frame = 1 #Este es el frame donde el vídeo empezará
    last_frame = i #Puedes cambiar i a el número del último frame que quieres generar. It will raise an error if that number of frames does not exist.

    fps = 30
    output_video = dir_output+file_title+'.mp4'

    frames = []

    for i in range(init_frame,last_frame): #
      filename = f"{dir_steps}/{i:04}.png"
      frames.append(Image.open(filename))

    from subprocess import Popen, PIPE
    p = Popen(['ffmpeg', '-y', '-f', 'image2pipe', '-vcodec', 'png', '-r', str(fps), '-i', '-', '-vcodec', 'libx264', '-r', str(fps), '-pix_fmt', 'yuv420p', '-crf', '13', '-preset', 'veryslow', output_video], stdin=PIPE)
    for im in tqdm(frames):
      im.save(p.stdin, 'PNG')
    p.stdin.close()
    p.wait()

    #---------------

    fin_vid = fin_out.replace('.png', '.mp4')
    op(c.ok, 'Video saved as', fin_vid)

  if repeat_index is repetitions:
    repeat_index = 1
  else:
    repeat_index += 1

op(c.title, '\nFIN.')